## HTML Extraction From Viki TV Website
**Author: Christopher Elliott**

**Team: KDramaQueen**

**March 24th 2019**

In [1]:
import bs4
from bs4 import BeautifulStoneSoup
import pandas as pd
import pickle
import wikipedia
import numpy as np
import requests
import json
from requests import get
from time import sleep
from itertools import chain

In [2]:
def extract_data(html):
    """ Extracts show data from viki TV HTML files
    and returns a dictonary with the Show Name as a key
    
    RETURNS: dict 
    """
    #Open File
    File = open(html+".html",encoding="Latin-1")
    soup = bs4.BeautifulSoup(File,'html.parser')
    titles = soup.find_all('a',"thumb-title strong")
    reviews = soup.find_all('span',"review-rating")
    num_reviews = soup.find_all("span", "caption info")
    
    if html == '32':
        del(titles[10])
    
    assert len(reviews) == len(titles)
    title_names ={}
    review = []
    
    #Data Extraction
    for i in range(0,len(titles)):
        # Title Extraction
        x = str(titles[i])
        t = x.find("</span>")
        x = x[t+len("</span>"):]
        n = x.find("\n")
        x = x[:n]
    
        # Review Extraction
        r = str(reviews[i])
        e = r.find('<span class="review-rating">')
        r=r[e+len('<span class="review-rating">'):]
        r = r[:3]
        r = float(r)
        
        # Number of Reviews Extraction
        b = num_reviews[i]
        b = str(b)
        k = b.find(">")
        b = b[k+2:]
        s = b.find(" ")
        b = b[:s]
        b= int(b)
    
        # Data Extraction
        title_names[x] = r,b
    File.close()
    return title_names

In [3]:
def build_dataframe():
    """Builds a pandas dataframe using the 
    HTML files in the directory and pickles the dataframe to the disk
    
    RETURNS: DataFrame
    NOTE: Will Create a file on local drive
    """
    print("Building Dataframe Part 1...")
    print("initializaing...")
    data = {}
    for i in range(0,38):
        if i%10==0:
            print('building ...') 
        i = str(i)
        data.update(extract_data(i))
            
    df = pd.DataFrame.from_dict(data,orient='index')
    df = df.reset_index()
    df.columns = ['Show','Viki Rating','Viki n_reviews']
    print("Saving Data Structure to Drive...")
    file = "Viki_HTML_data.pkl"
    pickle.dump(df,open(file,'wb'))   
    print("Complete!")
    return df

In [4]:
def build_korean_drama_data():
    data = {}
    for i in range(1,186):
        if i%25 == 0:
            print(int((i/186)*100),"Percent Complete")
        web = "https://www.koreandrama.org/page/"+str(i)
        webresponse = get(web)
        soup = bs4.BeautifulSoup(webresponse.text,'html.parser')
        posts = soup.find_all('div',"post")
        
        for i in posts:
            string = str(i)
            title_index = string.find("Title: ")
            string = string[title_index+len("Title: "):]
            title_index1a = string.find("/")
            string = string[title_index1a+1:]
            title_index2 = string.find('<')
            title = string[1:title_index2]
            title = title.strip()
            
            genre_index = string.find("Genre:")
            
            if genre_index>-1:
                string = string[genre_index+len("Genre:")+1:]
                genre_index2 = string.find("<")
                genre = string[:genre_index2].replace(" ","")
                genre = genre.split(",")
            else:
                genre = ["NaN"]
            
            net_index = string.find("Broadcast network: ")
            
            if net_index>-1:
                string = string[net_index+len("Broadcast network: "):]
                net_index2 = string.find("<")
                network = string[:net_index2]
            else:
                network = "NaN"
                
            data[title] = genre,network
            
    df = pd.DataFrame.from_dict(data,orient='index')
    df = df.reset_index()
    df.columns = ['Title','Genre','Network']
    df = df.replace('www.koreandrama.org/me-ri-dae-gus-attack-and-defense-battle/" rel="bookmark" title="Permanent Link to Me Ri, Dae Gu’s Attack and Defense Battle"> Me Ri, Dae Gu’s Attack and Defense Battle',"Me Ri, Dae Gu’s Attack and Defense Battle")
    df = df.replace('www.koreandrama.org/how-much-you-like-it/" rel="bookmark" title="Permanent Link to How Much You Like It"> How Much You Like It',"How Much You Like It")
    df = df.replace('www.koreandrama.org/golden-apple/" rel="bookmark" title="Permanent Link to Golden Apple"> Golden Apple'
,"Golden Apple")
    df = df.replace('프렌즈 / Friends',"Friends")
    df = df.loc[(df['Title']<'www.koreandrama.org/2018-sbs-drama-awards-winn')]
    df = df.replace("NaN",np.NaN)
            
    print("Saving Data Structure to Drive...")
    file = "KDrama_HTML_data.pkl"
    pickle.dump(df,open(file,'wb'))   
    print("Complete!")
    return df
     

In [5]:
def get_synopsis(data):
    dat = {}
    count = 0
    for i in data["Title"]:
        if count % 100 == 0:
            print(int((count/len(data['Title']))*100),"Percent Complete")
        j= i.replace(" ","-")
        web = "https://www.koreandrama.org/"+j+"/"
        try:
            webresponse = get(web)
        except:
            webresponse = get(web)
        soup = bs4.BeautifulSoup(webresponse.text,'html.parser')
        entry =soup.find_all('div',"entrytext")
        string = str(entry)
        n = len("Synopsis</strong></p><p>")
        m = len("</p><p><strong>Cast</strong>")
        index = string.find("Synopsis</strong></p><p>")
        lindex = string.find("</p><p><strong>Cast</strong>")
        string = string[index+n:lindex]
        dat[i] = string
        count +=1
        
    df = pd.DataFrame.from_dict(dat,orient='index')
    df = df.reset_index()
    df.replace('NaN',np.NaN)
    df.columns = ['Title','Summary']
    
    print("Saving Data Structure to Drive...")
    file = "KDData.pkl"
    pickle.dump(df,open(file,'wb'))   
    return df

In [6]:
def build_wiki_data(data):
    print("Building DataFrame Pt2")
    print("Starting...")
    t = {}
    count = 0
    d = len(data['Title'])
    for i in data['Title']:
        if count%100 == 0:
            print(str(count) + " done out of " +str(d))
        try:
            wiki =wikipedia.page(i+" South Korea")
            t[i] = wiki.summary,wiki.categories
        except:
            t[i]="NaN","NaN"
        count +=1
    
    df = pd.DataFrame.from_dict(t,orient='index')
    df = df.reset_index()
    df.replace('NaN',np.NaN)
    df.columns = ['Titles','Wiki Summary','Wiki Tags']
    
    print("Saving Data Structure to Drive...")
    file = "Wiki_data.pkl"
    pickle.dump(df,open(file,'wb'))   
    return df

In [7]:
data = build_korean_drama_data()

13 Percent Complete
26 Percent Complete
40 Percent Complete
53 Percent Complete
67 Percent Complete
80 Percent Complete
94 Percent Complete
Saving Data Structure to Drive...
Complete!


In [8]:
synopsis = get_synopsis(data)

0 Percent Complete
6 Percent Complete
13 Percent Complete
20 Percent Complete
27 Percent Complete
34 Percent Complete
40 Percent Complete
47 Percent Complete
54 Percent Complete
61 Percent Complete
68 Percent Complete
75 Percent Complete
81 Percent Complete
88 Percent Complete
95 Percent Complete
Saving Data Structure to Drive...


In [9]:
synopsis
data
df1 = pd.merge(data, synopsis, on='Title')
df1

,Title,Genre,Network,Summary
0,Witch’s Love,"[Comedy, Romance, Fantasy]","MBN, DramaX",
1,You Are a Gift,"[Melodrama, Romance]",SBS,"This drama is about a costume designer, Kong H..."
2,Let’s Marry,"[Drama, Romance]",MBC,
3,Possessed,"[Horror, Comedy]",OCN,"Hong Seo Jung (<a href=""https://star.koreandra..."
4,The Snow Queen,"[Romance, Drama]",NaN,"At 17, Tae Woong (Hyun Bin), a mathmatical gen..."
5,Prince Of Prince,"[Romance, Comedy, Fantasy]",NAVER tvcast,This Web drama is based on the Webtoon <a href...
6,Queen of Ambition,"[Melodrama, Romance]",SBS,"This drama is about a woman, Joo Da Hae (<a hr..."
7,Mom,"[Family, Melodrama]",NaN,"Yoon Jung Ae (<a href=""http://star.koreandrama..."
8,My Man’s Secret,"[Romance, Family, Melodrama]",KBS2,
9,Bulgurui Myeoneuri / Indomitable Daughters in Law,"[Family, drama]",MBC,The story of three generations of daughters-in...


In [11]:
wiki = build_wiki_data(df1)

Building DataFrame Pt2
Starting...
0 done out of 1466


/Users/sophiezhao/venv/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


100 done out of 1466
200 done out of 1466
300 done out of 1466
400 done out of 1466
500 done out of 1466
600 done out of 1466
700 done out of 1466
800 done out of 1466
900 done out of 1466
1000 done out of 1466
1100 done out of 1466
1200 done out of 1466
1300 done out of 1466
1400 done out of 1466
Saving Data Structure to Drive...


In [12]:
wiki = wiki.rename(columns={'Titles': 'Title'})
df = pd.merge(wiki, df1, on='Title')

In [13]:
df.to_csv(r"C:\Users\Chris\Desktop\Viki Tv HTML Extraction\Data_Set_New.csv")

In [15]:
df = pd.DataFrame.from_csv("Data_Set_New.csv")

/Users/sophiezhao/venv/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


FileNotFoundError: [Errno 2] File b'Data-Set-Final.csv' does not exist: b'Data-Set-Final.csv'